In [1]:
import findspark
findspark.init()
import pyspark

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [2]:
spark=SparkSession.builder.appName('Assignment_3').getOrCreate()

In [3]:
#READING DATASET 
data=(spark.read.option('header','true').csv('MoviesA3.csv', sep=";" , inferSchema=True)).persist()

In [4]:
data.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Length: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Actor: string (nullable = true)
 |-- Actress: string (nullable = true)
 |-- Director: string (nullable = true)
 |-- Popularity: integer (nullable = true)
 |-- Awards: string (nullable = true)
 |-- Image: string (nullable = true)



In [5]:
data.columns

['Year',
 'Length',
 'Title',
 'Genre',
 'Actor',
 'Actress',
 'Director',
 'Popularity',
 'Awards',
 'Image']

#  <font color='green'>Assignment 3 - Spark Data Frames and Clustering</font>

### Question # 1: Find the title, year and director of action films that won an award.

In [6]:
data.select('Genre').show(5)

+------+
| Genre|
+------+
|Comedy|
|Comedy|
|Horror|
|Action|
| Drama|
+------+
only showing top 5 rows



In [7]:
data.select('Title','Year','Director').filter((data.Genre=='Action') & (data.Awards=='Yes')).show()

+-----+----+--------+
|Title|Year|Director|
+-----+----+--------+
+-----+----+--------+



### Question # 2: For each award-winning actor, find the movies he acted it. Print the names of the movies and the director of the movie.

In [8]:
award_winning_actors= data.select('Actor').filter(data.Awards=='Yes')
data.select('Title','Director').filter(award_winning_actors.Actor==data.Actor).show()
#award_winning_actors.show(10)
#data.select('Title','Year','Director').filter((data.Director=='Action') & (data.Awards=='Yes')).show()

+--------------------+--------------------+
|               Title|            Director|
+--------------------+--------------------+
|Tie Me Up! Tie Me...|    Almod�var, Pedro|
|          High Heels|    Almod�var, Pedro|
|      Dead Zone, The|   Cronenberg, David|
|                Cuba|     Lester, Richard|
|      Days of Heaven|    Malick, Terrence|
|           Octopussy|          Glen, John|
|        Target Eagle|Loma, Jos� Antoni...|
|American Angels: ...|  Sebastian, Beverly|
|              Subway|         Besson, Luc|
|     Camille Claudel|      Nuytten, Bruno|
| Fanny and Alexander|     Bergman, Ingmar|
|Tragedy of a Ridi...|Bertolucci, Bernardo|
|     A Man & a Woman|     Lelouch, Claude|
|A Man & a Woman: ...|     Lelouch, Claude|
|Un Hombre y una M...|     Lelouch, Claude|
| Official Story, The|        Puenzo, Luiz|
|Lindbergh Kidnapp...|         Kulik, Buzz|
|           Blackmail|   Hitchcock, Alfred|
|      Donovan's Reef|          Ford, John|
|Tucker: The Man &...|Coppola, F

### Question # 3: Find the top 10 most popular movies that did not win an award.

In [9]:
data.select('Title','Popularity').orderBy((data.Popularity).desc()).filter(data.Awards=='No').show(10)

+--------------------+----------+
|               Title|Popularity|
+--------------------+----------+
|         Let It Ride|        88|
|     Great Race, The|        88|
|      New Year's Day|        88|
|        Final Notice|        88|
|   Fellini Satyricon|        88|
| Guilty by Suspicion|        88|
|   Time Machine, The|        88|
|           Raw Nerve|        88|
|Long Voyage Home,...|        88|
|           Class Act|        88|
+--------------------+----------+
only showing top 10 rows



### Question # 4: Find the 10 least popular movies that were released before 1980.

In [10]:
data.select('Title','Popularity').orderBy((data.Popularity).asc()).filter(data.Year<='1980').show(10)

+------------------+----------+
|             Title|Popularity|
+------------------+----------+
|      Desert Rider|      null|
| Bank on the Stars|      null|
|   White Lightning|      null|
|    Drop Kick, The|      null|
|           Shalako|         0|
|           Airport|         0|
|Shout at the Devil|         0|
|     Anna Christie|         0|
|      It's My Turn|         0|
| Tales of Tomorrow|         0|
+------------------+----------+
only showing top 10 rows



### Question # 5: Find the average length of the movies of each genre.

In [11]:
data.groupBy(data.Genre).avg('Length').alias('Average Length').show(10) # Can't use Avg sum or any other function before group by just like in sql writing this down to remember next time

+---------------+------------------+
|          Genre|       avg(Length)|
+---------------+------------------+
|          Crime|              66.0|
|        Romance|             127.0|
|      Adventure|             119.0|
|           null|             120.5|
|          Drama|113.30455259026688|
|            War|         116.90625|
|        Fantasy|             102.0|
|        Mystery|103.00990099009901|
|          Music|100.48780487804878|
|Science Fiction|106.47368421052632|
+---------------+------------------+
only showing top 10 rows



### Question # 6: Find the actor and actress pair who have acted in more than three Comedies together.

In [12]:
from pyspark.sql.functions import col
data.filter(data.Genre=='Comedy').groupBy('Actor','Actress').count().alias('Count').filter((col('count')>3) & (data.Actor.isNotNull() & data.Actress.isNotNull())).show()

+--------------+------------------+-----+
|         Actor|           Actress|count|
+--------------+------------------+-----+
|  Allen, Woody|     Keaton, Diane|    5|
|Tracy, Spencer|Hepburn, Katharine|    6|
+--------------+------------------+-----+



### Question # 7: Find the names of actors who acted in movies of both ‘Comedy’ and ‘Drama’ Genre.

In [13]:
data.select('Actor').filter(data.Genre=='Comedy').distinct().intersect(data.select('Actor').filter(data.Genre=='Drama').distinct()).show()

+--------------------+
|               Actor|
+--------------------+
|      Boyer, Charles|
|      Taylor, Robert|
|        Lemmon, Jack|
|       Aiello, Danny|
|      Stewart, James|
|    Malmsten, Birger|
|         Taylor, Rod|
|       Russell, Kurt|
|     Hopkins, Harold|
|      Beatty, Warren|
|       Irons, Jeremy|
|        Fonda, Henry|
|      Ford, Harrison|
|   Depardieu, G�rard|
|     Eastwood, Clint|
|      Howard, Arliss|
|        Cleese, John|
|Lambert, Christopher|
|        Johnson, Ben|
|       Connery, Sean|
+--------------------+
only showing top 20 rows



### Question # 8: Find the names of actors who acted in movies of both ‘Comedy’ or ‘Drama’ Genre.

In [14]:
data.select('Actor').filter(data.Genre=='Comedy').distinct().union(data.select('Actor').filter(data.Genre=='Drama').distinct()).show()

+-----------------+
|            Actor|
+-----------------+
|  Matthau, Walter|
|Keillor, Garrison|
|Gauthier, Vincent|
|   Boyer, Charles|
|Finneran, Siohban|
|     Keach, James|
|Nagase, Masatoshi|
|   Taylor, Robert|
|    Mandel, Howie|
|     Chase, Chevy|
|   Wilson, George|
|      Arkin, Alan|
|     Lemmon, Jack|
|Banderas, Antonio|
|     Most, Donald|
|    Jaglom, Henry|
|    Rey, Fernando|
|    Aiello, Danny|
|     Reiner, Carl|
|     Cooper, Gary|
+-----------------+
only showing top 20 rows



### Question # 9: Find the names of actors who did not acted in any ‘Comedy’.

In [15]:
data.select('Actor').filter(data.Genre!='Comedy').distinct().show()

+--------------------+
|               Actor|
+--------------------+
|      Din, Ayub Khan|
|        Ferrer, Jose|
|       Harrison, Rex|
|      McCleery, Gary|
|         Busey, Gary|
|      Boyer, Charles|
|       Kime, Jeffrey|
|     McDowall, Roddy|
|          Race, Hugo|
|Luckinbill, Laurence|
|      Taylor, Robert|
|       Karyo, Tcheky|
|          Webb, John|
|         Ganus, Paul|
|       Longden, John|
|       Fisher, Eddie|
|        Keach, Stacy|
|   Bergen, Robert D.|
|    Sj�str�m, Victor|
|         Morrow, Vic|
+--------------------+
only showing top 20 rows



### Question # 10: Find each actor, find the mean, max, min ranking of his movies.

In [16]:
from pyspark.sql.functions import mean as _mean, col
from pyspark.sql import functions as f
data.groupBy("Actor").agg(_mean("Popularity") ,f.max("Popularity") ,f.min("Popularity")).filter(data.Actor.isNotNull()).orderBy((data.Actor).asc()).show()

+--------------------+-----------------+---------------+---------------+
|               Actor|  avg(Popularity)|max(Popularity)|min(Popularity)|
+--------------------+-----------------+---------------+---------------+
|        Abel, Alfred|             49.0|             49|             49|
|  Abraham, F. Murray|              6.0|              6|              6|
|    Adolphson, Edvin|             49.0|             72|             26|
|       Aherne, Brian|             57.0|             57|             57|
|     Ahlstedt, B�rje|             81.0|             81|             81|
|       Aiello, Danny|             12.5|             20|              5|
|         Akan, Tarik|             53.0|             53|             53|
|    Albaic�n, Rafael|             55.0|             55|             55|
|      Albert, Edward|             82.0|             82|             82|
|          Alda, Alan|             32.5|             53|             12|
|         Allen, Bill|             75.0|           

### Question # 11: List the number of movies released in each decade starting from 1960’s.

In [17]:
row1 = data.agg({"Year": "max"}).collect()[0]
print('Maximum Year to Determine total Decade is :',max(row1))
print("")
dataCollect=data.rdd.toLocalIterator()
first_decade=0
second_decade=0
third_decade=0
fourth_decade=0

for row in dataCollect:
    if (row['Year']>=1960 and row['Year']<=1969):
        first_decade=first_decade+1
    elif (row['Year']>=1970 and row['Year']<=1979):
        second_decade=second_decade+1
    elif (row['Year']>=1980 and row['Year']<=1989):
        third_decade=third_decade+1
    elif (row['Year']>=1990 and row['Year']<=1999):
        fourth_decade=fourth_decade+1

print("Number of movies in Decade Between 1960-1969 is : ",first_decade)
print("Number of movies in Decade Between 1970-1979 is : ",second_decade)
print("Number of movies in Decade Between 1980-1989 is : ",third_decade)
print("Number of movies in Decade Between 1990-1999 is : ",fourth_decade)
print("")
print("To Prove this is correct we check by following:-")
print("The sum of all decades = ",first_decade+second_decade+third_decade+fourth_decade)
print("The sum through pyspark query = ",data.filter(data.Year>=1960).count())


Maximum Year to Determine total Decade is : 1997

Number of movies in Decade Between 1960-1969 is :  154
Number of movies in Decade Between 1970-1979 is :  244
Number of movies in Decade Between 1980-1989 is :  607
Number of movies in Decade Between 1990-1999 is :  345

To Prove this is correct we check by following:-
The sum of all decades =  1350
The sum through pyspark query =  1350


### Question # 12: Find the number of movies released in each year.

In [18]:
data.groupBy("Year").count().show()

+----+-----+
|Year|count|
+----+-----+
|1959|    9|
|1990|  105|
|1975|   23|
|1977|   29|
|1924|    3|
|1974|   23|
|1927|    3|
|1955|   20|
|1978|   18|
|1925|    1|
|1961|   14|
|1942|    6|
|1944|    4|
|1939|   11|
|1952|   10|
|1956|   15|
|1934|    4|
|1988|   96|
|1997|    1|
|1968|   20|
+----+-----+
only showing top 20 rows



### Question # 13: Find the number of movies released in each year of each genre. Consider only the movies with length greater than 100 minutes.

In [19]:
data.filter(data.Length>100).groupBy("Year","Genre").count().show()

+----+---------------+-----+
|Year|          Genre|count|
+----+---------------+-----+
|1989|         Action|   10|
|1988|         Comedy|   10|
|1966|          Drama|    6|
|1985|         Comedy|    6|
|1957|        Western|    1|
|1928|          Drama|    2|
|1970|        Western|    2|
|1975|          Drama|    5|
|1969|         Action|    1|
|1940|         Action|    1|
|1939|        Western|    3|
|1987|         Comedy|   13|
|1982|Science Fiction|    1|
|1951|            War|    1|
|1965|          Music|    1|
|1939|         Comedy|    1|
|1964|          Music|    1|
|1987|            War|    1|
|1962|         Action|    1|
|1966|        Mystery|    3|
+----+---------------+-----+
only showing top 20 rows



### Question # 14: Sort the movie’s release before 1990 by the title.

In [20]:
data.select('Title').filter(data.Year<1990).sort(data.Title).show()

+--------------------+
|               Title|
+--------------------+
|2001: A Space Ody...|
|             48 Hrs.|
|               8 1/2|
|A Big Hand for th...|
|  A Child Is Waiting|
|A Chorus Line, Th...|
|  A Clockwork Orange|
|A Coeur Joie, (He...|
|   A Cry in the Dark|
|  A Dry White Season|
|      A Fine Madness|
| A Fish Called Wanda|
|A Fistful of Dollars|
|     A Guy Named Joe|
|    A Lesson in Love|
|A Little Night Music|
|     A Man & a Woman|
|A Man & a Woman: ...|
|A Man for All Sea...|
|    A Matter of Time|
+--------------------+
only showing top 20 rows



### Question # 15: Find the movies with long titles. A movie title is considered long if it is greater than 50 alphabets.

In [21]:
from pyspark.sql import functions as f
data.filter(f.length(data.Title)>50).select('Title').show()

+--------------------+
|               Title|
+--------------------+
|Fawlty Towers, Go...|
|Unnamable II, The...|
|Industrial Sympho...|
+--------------------+

